In [36]:
##============ Dependencies and libraries ============##
from dotenv import load_dotenv
import os

import requests
from collections import Counter

import pandas as pd
import time
import json

import re
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')
from deep_translator import GoogleTranslator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import praw


# Load environment variables from the .env file
load_dotenv()

# Retrieve credentials from environment variables
REDDIT_APP_ID = os.getenv('REDDIT_APP_ID')
REDDIT_SECRET = os.getenv('REDDIT_SECRET')


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/cezarykubinski/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Keyword preparation

In [37]:
##============ Load json dictionary ============##

# Path to the JSON file
file_path = "waterCompanies.json"

# Load JSON data into a Python dictionary
with open(file_path, "r", encoding="utf-8") as file:
    json_data = json.load(file)

# Access the loaded JSON data
print(json_data)


{'company_names': {'major_water_companies': {'American_Water_Works': {'en': ['American Water Works', 'AWK'], 'es': ['American Water Works']}, 'Veolia_Environnement': {'en': ['Veolia Environnement', 'VE'], 'fr': ['Veolia Environnement'], 'es': ['Veolia Medio Ambiente', 'Veolia'], 'de': ['Veolia Umwelt']}, 'Essential_Utilities': {'en': ['Essential Utilities', 'Aqua America', 'WTRG'], 'es': ['Servicios Esenciales']}, 'Xylem_Inc': {'en': ['Xylem Inc.', 'XYL'], 'fr': ['Xylem France'], 'de': ['Xylem Deutschland']}, 'California_Water_Service_Group': {'en': ['California Water Service Group', 'CWT'], 'es': ['Grupo de Servicios de Agua de California']}, 'Suez_SA': {'fr': ['Suez Environnement', 'SEV'], 'es': ['Suez Medio Ambiente'], 'de': ['Suez Umwelt']}, 'Global_Water_Resources': {'en': ['Global Water Resources', 'GWRS'], 'es': ['Recursos Hídricos Globales']}}}, 'general_industry': {'water_supply': {'en': ['water utility', 'water supply', 'municipal water', 'public utility', 'water management',

In [29]:
##============ Extract keywords ============##


# Function to extract all keywords from the JSON dictionary
def extract_keywords(keywords_dict):
    keywords = []
    
    # Extract company names
    for company_group in keywords_dict.get('company_names', {}).values():
        for company in company_group.values():
            for lang_keywords in company.values():
                keywords.extend(lang_keywords)
    
    # Extract general industry terms
    for category in keywords_dict.get('general_industry', {}).values():
        for lang_keywords in category.values():
            keywords.extend(lang_keywords)

    # Extract event and issue-related terms
    for category in keywords_dict.get('events_and_issues', {}).values():
        for lang_keywords in category.values():
            keywords.extend(lang_keywords)

    # Extract financial and market sentiment-related terms
    for category in keywords_dict.get('financial_and_market_sentiment', {}).values():
        for lang_keywords in category.values():
            keywords.extend(lang_keywords)

    # Extract technical and innovation-related terms
    for category in keywords_dict.get('technical_and_innovation', {}).values():
        for lang_keywords in category.values():
            keywords.extend(lang_keywords)

    # Extract regional-specific terms (optional)
    for region in keywords_dict.get('regional', {}).values():
        for lang_keywords in region.get('regional_terms', []):
            keywords.append(lang_keywords)

    # Remove duplicates
    return list(set(keywords))

keywords = extract_keywords(json_data)

print(keywords)


['GWRS', 'conservación del agua', 'Wassereinsparung', 'water sector investment', 'Actions de services publics d’eau', 'Unternehmensführung)', 'Wasserwerk', 'sequía', 'lead in water', 'política ambiental', 'environmental regulations', 'automatización del agua', 'changement climatique', 'conservation funds', 'Veolia', 'wastewater treatment', 'Umweltpolitik', 'automatisation de l’eau', 'Wasserknappheit', 'cuota de mercado', 'investissement dans le secteur de l’eau', 'eau numérique', 'market share', 'regulaciones ambientales', 'ingresos', 'Recursos Hídricos Globales', 'Aqua America', 'gestión del agua', 'Abwasserbehandlung', 'Wasserinfrastruktur', 'Gouvernance)', 'California Water Service Group', 'cambio climático', 'water crisis', 'water supply', 'infrastructures hydrauliques', 'IoT im Wasser', 'المياه', 'water management', 'suministro de agua', 'efficacité hydrique', 'clima extremo', 'microplásticos', 'inversión en infraestructura de agua', 'water scarcity', 'réglementations environnemen

In [30]:
##============ Enrich keywords with synonyms from NLTK ============##

def expand_keywords(keywords):
    expanded_keywords = set(keywords)
    for keyword in keywords:
        for syn in wordnet.synsets(keyword):
            for lemma in syn.lemmas():
                expanded_keywords.add(lemma.name().replace('_', ' '))
    return list(set(expanded_keywords))

expanded_keywords = expand_keywords(keywords)

print(expanded_keywords)



['Wassereinsparung', 'lucre', 'Actions de services publics d’eau', 'sequía', 'lead in water', 'automatización del agua', 'conservation funds', 'Umweltpolitik', 'automatisation de l’eau', 'Wasserknappheit', 'cuota de mercado', 'taxation', 'investissement dans le secteur de l’eau', 'deluge', 'eau numérique', 'market share', 'gestión del agua', 'Abwasserbehandlung', 'water crisis', 'infrastructures hydrauliques', 'quintal', 'short hundredweight', 'profits', 'suministro de agua', 'microplásticos', 'inversión en infraestructura de agua', 'water scarcity', 'réglementations environnementales', 'leak detection', 'crecimiento', 'Saudi Arabia', 'turn a profit', 'Umweltvorschriften', 'water automation', 'sewage treatment', 'flood', 'recursos hídricos', 'Essential Utilities', 'sostenibilidad', 'approvisionnement en eau', 'durabilité', 'Investitionen im Wassersektor', 'and regional water initiatives', 'Veolia Environnement', 'servicios públicos', 'part de marché', 'loi sur l’eau propre', 'digitales

In [34]:
##============ Enrich keywords with translations from deep ============##

translated_keywords = {lang: [GoogleTranslator(source='auto', target=lang).translate(word) for word in keywords]
                       for lang in ['fr', 'de', 'es', 'zh-CN', 'pt', 'pa']}

# Combine all keywords and translations into a single list
print(translated_keywords)

unified_keywords = keywords + [word for lang in translated_keywords.values() for word in lang]
unified_keywords = list(set(unified_keywords))

print(unified_keywords)

{'fr': ['COURS', "conservation de l'eau", "Économie d'eau", "Investissement dans le secteur de l'eau", 'Actions de services publics d’eau', "gestion d'entreprise)", 'Aqueduc', 'sécheresse', "plomb dans l'eau", 'politique environnementale', 'réglementation environnementale', "automatisation de l'eau", 'changement climatique', 'fonds de conservation', 'Véolia', 'traitement des eaux usées', 'Politique environnementale', 'automatisation de l’eau', "Pénurie d'eau", 'part de marché', 'investissement dans le secteur de l’eau', 'eau numérique', 'Part de marché', 'réglementation environnementale', 'revenu', 'Ressources mondiales en eau', 'Aqua Amérique', "gestion de l'eau", 'Traitement des eaux usées', 'Infrastructures hydrauliques', 'Gouvernance)', "Groupe de services d'eau de Californie", 'changement climatique', "crise de l'eau", 'approvisionnement en eau', 'infrastructures hydrauliques', "L'IoT dans l'eau", 'Eau', "gestion de l'eau", 'approvisionnement en eau', 'efficacité hydrique', 'condi

In [38]:
print(f"Keywords number: {len(keywords)}")
print(f"Extended keywords number: {len(expanded_keywords)}")
print(f"Translated keywords number: {len(unified_keywords)}")


Keywords number: 231
Extended keywords number: 288
Translated keywords number: 805


### Data scraping

In [40]:
##============ Find relevant subreddits from pushshift ============##

def fetch_subreddits_for_keywords(keywords, after=None, before=None, size=100):
    base_url = "https://api.pushshift.io/reddit/search/submission/"
    subreddit_counter = Counter()
    
    for keyword in keywords:
        params = {
            "q": keyword,
            "after": after,
            "before": before,
            "size": size,
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            posts = response.json().get("data", [])
            for post in posts:
                subreddit_counter[post["subreddit"]] += 1
        else:
            print(f"Error fetching data for keyword {keyword}: {response.status_code} {response.text}")
        
        time.sleep(5) 
    
    return subreddit_counter.most_common()

keywords = unified_keywords
subreddits = fetch_subreddits_for_keywords(keywords, after="2023-01-01", size=100)
print("Top relevant subreddits:")
for subreddit, count in subreddits:
    print(f"{subreddit}: {count} mentions")


Error fetching data for keyword 益处: 403 {"detail":"Not authenticated"}
Error fetching data for keyword Emiratos Árabes Unidos: 403 {"detail":"Not authenticated"}
Error fetching data for keyword Réglementation environnementale: 403 {"detail":"Not authenticated"}
Error fetching data for keyword ਪਾਣੀ ਦਾ ਲੂਣੀਕਰਨ: 403 {"detail":"Not authenticated"}
Error fetching data for keyword Soziales und Governance): 403 {"detail":"Not authenticated"}


KeyboardInterrupt: 

In [ ]:
##============ Test Reddit API ============##


import praw
import pandas as pd
import time

# Authenticate with Reddit API
reddit = praw.Reddit(
    client_id=REDDIT_APP_ID,       # Replace with your client ID
    client_secret=REDDIT_SECRET,  # Replace with your client secret
    user_agent="WaterDataCollector",    # A custom user-agent
)

# Define Subreddits, Keywords, and Parameters
subreddits = [
    "worldnews", "news", "geopolitics", "environment", "sustainability", "climate", "globalclimatechange", "conservation", "drought", "wallstreetbets", "StockMarket",
    "investing", "stocks", "Infrastructure", "India", "Australia", "Europe", "engineering", "UrbanPlanning", "UnitedKingdom", "MiddleEast", "AgTech", "foodsecurity", "GlobalHealth", "desalination"
]

# Define Search Function with comments
def search_reddit_with_comments(subreddits, keywords, limit=50):
    collected_data = []
    for subreddit in subreddits:
        for keyword in keywords:
            print(f"Searching '{keyword}' in r/{subreddit}...")
            try:
                for post in reddit.subreddit(subreddit).search(keyword, sort="new", time_filter="all", limit=limit): # Search for specific time periods using the time_filter parameter ('all', 'day', 'week', 'month', 'year')
                    post.comments.replace_more(limit=0)  # Load all comments
                    comments = [comment.body for comment in post.comments.list()]  # Extract all comments

                    collected_data.append({
                        "Subreddit": subreddit,
                        "Keyword": keyword,
                        "Title": post.title,
                        "Author": post.author.name if post.author else "N/A",
                        "Upvotes": post.score,
                        "Comments": post.num_comments,
                        "Created At": time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(post.created_utc)),
                        "URL": post.url,
                        "Content": post.selftext[:500],  # Truncate content for storage
                        "Top Comments": comments[:10]  # Collect up to 10 top comments
                    })
            except Exception as e:
                print(f"Error fetching data from r/{subreddit}: {e}")
    return collected_data

# Execute Search and Save Data
data = search_reddit(subreddits, keywords, limit=100)
df = pd.DataFrame(data)
df.to_csv("reddit_data.csv", index=False)

print(f"Collected {len(df)} posts. Data saved to 'reddit_data.csv'.")

# Load the CSV file into a DataFrame
reddit_data = pd.read_csv('reddit_data.csv')

reddit_data


Searching '益处' in r/worldnews...
Searching 'Emiratos Árabes Unidos' in r/worldnews...
Searching 'Réglementation environnementale' in r/worldnews...
Searching 'ਪਾਣੀ ਦਾ ਲੂਣੀਕਰਨ' in r/worldnews...
Searching 'Soziales und Governance)' in r/worldnews...
Searching 'Systèmes d'eau intelligents' in r/worldnews...
Searching 'gestão de água' in r/worldnews...
Searching 'conservation funds' in r/worldnews...
Searching '水服务行动' in r/worldnews...
Searching '威立雅环境' in r/worldnews...
Searching 'Umweltpolitik' in r/worldnews...
Searching 'automatisation de l’eau' in r/worldnews...
Searching 'Reservas de agua' in r/worldnews...
Searching 'ਡੀਸਲੀਨੇਸ਼ਨ' in r/worldnews...
Searching 'eau numérique' in r/worldnews...
Searching 'market share' in r/worldnews...
Searching 'Tratamiento de aguas residuales' in r/worldnews...
Searching 'Xilema Alemania' in r/worldnews...
Searching 'ਪਾਣੀ ਦਾ ਕਾਨੂੰਨ' in r/worldnews...
Searching 'ਬੁਨਿਆਦੀ ਢਾਂਚਾ ਖਰਚ' in r/worldnews...
Searching 'Services essentiels' in r/worldnews...
Sea

### Data procesing

In [ ]:
##============ Data Preprocessing ============##

def clean_text(text):
    text = re.sub(r"http\S+|www\S+", "", text)  # Remove URLs
    text = re.sub(r"[^A-Za-z0-9\s]", "", text)  # Remove special characters
    return text.lower()


In [ ]:
##============ Data Filtering ============##

filtered_data = df[(df['Upvotes'] > 50) & (df['Comments'] > 10)]


In [ ]:
##============ Sentiment Analysis ============##
# Compare sentiment trends before and after major events

analyzer = SentimentIntensityAnalyzer()
sentiment = analyzer.polarity_scores(text)